# Patch download - AgSal

## Modules

In [49]:
# Utils
import os
import requests
import winsound
import math

# Data manipulation
import pandas as pd
from datetime import timedelta, datetime

# Google Earth Engine
import ee

## Enviromental variables

In [17]:
DATA_ROOT_PATH = "../data"
LEAKS_46 = DATA_ROOT_PATH + "/raw/agsal_46.xlsx"
LEAKS_49 = DATA_ROOT_PATH + "/raw/agsal_49.xlsx"
SAVE_CLEAN_DATA = DATA_ROOT_PATH + "/clean"

## Doc

### Functions

In [57]:
# Get image from image collection
def get_image(start, end, poi_leak):
    
    if isinstance(start, str) == False:
        start = str(start)
        end = str(end)
    
    collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
    .filterBounds(poi_leak)\
    .filterDate(ee.Date(start), ee.Date(end))\
    .sort("system:time_start", opt_ascending = False)\
    .sort("CLOUDY_PIXEL_PERCENTAGE")

    img = collection.first()

    print("Date of selected image (Sentinel): ", ee.Date(img.get("system:time_start")).format("yyyy-MM-dd").getInfo(),
          "\nSentinel images found:", collection.size().getInfo(),
          "\nCloud %: ", img.get("CLOUDY_PIXEL_PERCENTAGE").getInfo()) 
    
    return img  

In [58]:
# Define the bands to select and the patch size (radius in meters respect to leak point)
def bands_clip_image(image, buffer_size = 100, bands = ["B4", "B3", "B2"]):
    # Clip image
    image = image.clip(poi.buffer(buffer_size).bounds(proj = "EPSG:32613", maxError = 0.001))

    # Select bands
    image = image.select(bands)

    return image

In [59]:
# Download patch image
def download_image(image, path, date_label):
    url = image.getDownloadURL(
        {
        "scale": 10,
        "crs": "EPSG:32613",
        "fileFormat": "GeoTIFF",
        "maxPixels": 1e13
        }
    )

    r = requests.get(url, allow_redirects = True)
    open(path + "S2" + "_" + date_label + ".zip", "wb").write(r.content)
    print("Download complete")

In [60]:
def get_landsat_images(start, end, poi_leak, buffer_regression):

    poi_regression = poi_leak.buffer(buffer_regression).bounds(proj = "EPSG:32613", maxError = 0.001)
 
    try:

        if isinstance(start, str) == False:
            start = str(start)
            end = str(end)
        
        selected_Landsat_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
                                        .filterBounds(poi_leak)\
                                        .filterDate(ee.Date(start), ee.Date(end))\
                                        .sort("CLOUD_COVER")

        img_landsat = selected_Landsat_collection.first().clip(poi_regression)

        print("Date of selected image (Landsat):", ee.Date(img_landsat.get("system:time_start")).format("yyyy-MM-dd").getInfo(),
            "\nLandsat images found: ", selected_Landsat_collection.size().getInfo(),
            "\nCloud %: ", img_landsat.get("CLOUD_COVER").getInfo())
    
        if selected_Landsat_collection.size().getInfo() == 0:
            print("THERES NO LANDSAT IMAGES FOR THE SELECTED DATES")
            print("Stopped in rep", rep)

    except:
        start = start + timedelta(days = 5)

        if isinstance(start, str) == False:
            start = str(start)
            end = str(end)

        selected_Landsat_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
                                        .filterBounds(poi_leak)\
                                        .filterDate(ee.Date(start), ee.Date(end))\
                                        .sort("CLOUD_COVER")

        img_landsat = selected_Landsat_collection.first().clip(poi_regression)

        print("Date of selected image (Landsat):", ee.Date(img_landsat.get("system:time_start")).format("yyyy-MM-dd").getInfo(),
            "\nLandsat images found: ", selected_Landsat_collection.size().getInfo(),
            "\nCloud %: ", img_landsat.get("CLOUD_COVER").getInfo())
    
        if selected_Landsat_collection.size().getInfo() == 0:
            print("THERES NO LANDSAT IMAGES FOR THE SELECTED DATES")

    return img_landsat

In [61]:
# Create compund bands in order to perform linear regression
def lst_regression(landsat_image, sentinel_image, poi_leak, buffer_size, buffer_regression = 500, error = True):
    ndvi_landsat = landsat_image.normalizedDifference(["SR_B5", "SR_B4"]).rename("ndvi")
    ndwi_landsat = landsat_image.normalizedDifference(["SR_B3", "SR_B5"]).rename("ndwi")
    ndbi_landsat = landsat_image.normalizedDifference(["SR_B6", "SR_B5"]).rename("ndbi")
    lst_landsat_30m = landsat_image.select("ST_B10").rename("Landsat_LST_30m")

    ndvi_sentinel = sentinel_image.normalizedDifference(["B8", "B4"]).rename("s2_ndvi")
    ndwi_sentinel = sentinel_image.normalizedDifference(["B3", "B11"]).rename("s2_ndwi")
    ndbi_sentinel = sentinel_image.normalizedDifference(["B11", "B8"]).rename("s2_ndbi")

    # Poi for regression, this increases n
    poi_regression = poi_leak.buffer(buffer_regression).bounds(proj = "EPSG:32613", maxError = 0.001)

    # Linear regression
    bands = ee.Image(1).addBands(ndvi_landsat).addBands(ndbi_landsat).addBands(ndwi_landsat).addBands(lst_landsat_30m).rename(["constant", "ndvi", "ndbi", "ndwi", "lst"])

    img_landsat_regression = bands.reduceRegion(
        reducer = ee.Reducer.linearRegression(4, 1),
        geometry = poi_regression,
        scale = 30,
        maxPixels = 1e13
    )

    # Get coefficients of linear regression
    coefList2 = ee.Array(img_landsat_regression.get("coefficients")).toList()
    intercept2 = ee.Image(ee.Number(ee.List(coefList2.get(0)).get(0))).reproject(crs = "EPSG:32613")
    slopeNDVI2 = ee.Image(ee.Number(ee.List(coefList2.get(1)).get(0))).reproject(crs = "EPSG:32613")
    slopeNDBI2 = ee.Image(ee.Number(ee.List(coefList2.get(2)).get(0))).reproject(crs = "EPSG:32613")
    slopeNDWI2 = ee.Image(ee.Number(ee.List(coefList2.get(3)).get(0))).reproject(crs = "EPSG:32613")

    # Predict LST with Landsat info 
    LST_model = intercept2.add(slopeNDVI2.multiply(ndvi_landsat))\
                .add(slopeNDBI2.multiply(ndbi_landsat))\
                .add(slopeNDWI2.multiply(ndwi_landsat))

    # Get residuals
    residuals = lst_landsat_30m.subtract(LST_model)

    # Predict LST with Sentinel info (downscaled image)
    lst_landsat_10m_final = ee.Image(intercept2).add(slopeNDVI2.multiply(ndvi_sentinel))\
                                                .add(slopeNDBI2.multiply(ndbi_sentinel)).add(slopeNDWI2.multiply(ndwi_sentinel))\
                                                .clip(poi_leak.buffer(buffer_size).bounds(proj = "EPSG:32613", maxError = 0.001))

    if error == True:
        
        # Compute number of pixels in the image (n)
        n = landsat_image.select("ST_B10").reduceRegion(
            reducer = ee.Reducer.count(),
            scale = 30,
            maxPixels = 1e13
        ).getInfo()["ST_B10"]

        # Compute Root Mean Squared Error
        rmse = math.sqrt(residuals.select("Landsat_LST_30m").pow(2).reduceRegion(
            reducer = ee.Reducer.sum(),
            scale = 30,
            maxPixels = 1e13
        ).getInfo()["Landsat_LST_30m"]/n)

    return (lst_landsat_10m_final, rmse)

In [62]:
def add_lst10m_band(sentinel_image, lst_image):
    
    return sentinel_image.addBands(lst_image.rename("LST_10m"))

### Variables

In [74]:
important_raw_columns = ["NO. ORDEN", "FECHA DE CREACION", "FECHA TERMINACION", "TIPO DE ORDEN", "LATITUD", "LONGITUD"]
rename_raw_columns = {"NO. ORDEN": "id", 
                      "FECHA DE CREACION": "fecha_de_i",
                      "FECHA TERMINACION": "fechalegal",
                      "TIPO DE ORDEN": "tipo",
                      "LONGITUD": "x",
                      "LATITUD": "y"}

date_format = "%d-%m-%Y %H:%M:%S"

# Patch download
bands = ["B4", "B3", "B2", "B1", "B5", "B6", "B7", "B8", "B8A", "B9", "B11", "B12", "WVP"]
i = 0
download_path = DATA_ROOT_PATH + "/patches_raw_agsal/leak/"
patch_size = 100
index_progress = []
id = []
sentinel_img_date = []
landsat_img_date = []
date_diff = []
date_diff_leak = []
sentinel_cloud = []
landsat_cloud = []
rmse = []

### Load data

In [7]:
leaks_46_raw = pd.read_excel(LEAKS_46)
leaks_49_raw = pd.read_excel(LEAKS_49)

### EDA

In [5]:
leaks_46_raw.shape

(2030, 22)

In [6]:
leaks_46_raw["TIPO DE ORDEN"].value_counts()

FC.FUGA EN CALLE          843
FB.FUGA EN BANQUETA       461
FP.FUGA EN PIE DERECHO    235
TE.TRABAJOS ESPECIALES    228
TB.TUBO ROTO              208
CT.CAMBIO DE TOMA          55
Name: TIPO DE ORDEN, dtype: int64

In [8]:
leaks_49_raw.shape

(2176, 18)

In [9]:
leaks_49_raw["TIPO DE ORDEN"].value_counts()

FC.FUGA EN CALLE          1085
FB.FUGA EN BANQUETA        555
TB.TUBO ROTO               242
TE.TRABAJOS ESPECIALES     145
FP.FUGA EN PIE DERECHO      93
CT.CAMBIO DE TOMA           51
FL.FUGA NO VISIBLE           5
Name: TIPO DE ORDEN, dtype: int64

In [12]:
leaks_46_raw

,NO. ORDEN,FECHA DE CREACION,QUEJA AQUACIS,FECHA ASIGNACION,FECHA REALIZACION,FECHA TERMINACION,AREA,ZONA,BRIGADA,NUMERO CONTRATO MARCO,...,CLIENTE,TIPO DE ORDEN,TIPO DE PROYECTO,PROYCTO,POZO,CENTRO DE COSTOS,LATITUD,LONGITUD,TIPO DE ORDEN.1,DIRECCION
0,2505520,18/12/2023 14:50:44,2469457.0,20/12/2023 10:35:44,20/12/2023 12:44:22,16/01/2024 12:59:05,EFICIENCIA FISICA,ORIENTE,BR JUAN GOMEZ,4.600001e+09,...,7-ELEVEN MEXICO,FP.FUGA EN PIE DERECHO,NaN,NaN,NaN,74000,25.438982,-100.978640,NaN,JOSE MARIA LA FRAGUA 2530 GUANAJU...
1,2505513,18/12/2023 14:42:55,2469451.0,20/12/2023 08:36:06,20/12/2023 12:26:16,16/01/2024 12:58:41,EFICIENCIA FISICA,ORIENTE,BR JUAN GOMEZ,4.600001e+09,...,MOISES HERNANDEZ VALDES,FC.FUGA EN CALLE,NaN,NaN,NaN,74000,25.438315,-100.978401,NaN,DOLORES HIDALGO 220 GUANAJUATO ORIENTE
2,2505512,18/12/2023 14:42:55,2469450.0,20/12/2023 16:36:58,27/12/2023 15:47:00,16/01/2024 12:57:35,EFICIENCIA FISICA,ORIENTE,BR JOSE RODRIGUEZ,4.600001e+09,...,ALFA REYES VDA DEL BOSQUE,FB.FUGA EN BANQUETA,NaN,NaN,NaN,74000,25.454452,-101.009695,NaN,CORTAZAR 114 GUANAJUATO ORIENTE
3,2505509,18/12/2023 14:38:48,2469444.0,20/12/2023 08:36:43,20/12/2023 12:28:49,16/01/2024 12:57:07,EFICIENCIA FISICA,ORIENTE,BR JUAN GOMEZ,4.600001e+09,...,ALBERTO PECINA MORENO,FP.FUGA EN PIE DERECHO,NaN,NaN,NaN,74000,25.438911,-100.978670,NaN,DOLORES HIDALGO 124 GUANAJUATO ORIENTE
4,2501253,12/12/2023 12:28:43,2468338.0,13/12/2023 11:35:44,20/12/2023 11:42:28,16/01/2024 12:38:31,EFICIENCIA FISICA,PONIENTE,BR JUAN GOMEZ,4.600001e+09,...,JOVITA GUTIERREZ AGUILERA,FC.FUGA EN CALLE,NaN,NaN,NaN,74000,25.380674,-101.014129,NaN,PASEO DE LAS BEGONIAS 465 PARQUES DE LA C...


### Transform

In [14]:
# Select important columns and rename it
leaks_46_clean = leaks_46_raw[important_raw_columns].rename(columns=rename_raw_columns)
leaks_49_clean = leaks_49_raw[important_raw_columns].rename(columns=rename_raw_columns)

In [34]:
# Extract leak type
leaks_46_clean["tipo_cod"] = leaks_46_clean["tipo"].str.extract("([A-Z]+)(?<=.)")
leaks_49_clean["tipo_cod"] = leaks_49_clean["tipo"].str.extract("([A-Z]+)(?<=.)")

In [70]:
# Join both dfs
leaks_clean = pd.concat([leaks_46_clean, leaks_49_clean]).drop_duplicates()

In [72]:
# Fix date format
leaks_clean["fecha_de_i"] = leaks_clean["fecha_de_i"].str.replace("/", "-")
leaks_clean["fechalegal"] = leaks_clean["fechalegal"].str.replace("/", "-")

#### Save clean dfs

In [75]:
leaks_46_clean.to_csv(SAVE_CLEAN_DATA + "/agsal_46_clean.csv")
leaks_49_clean.to_csv(SAVE_CLEAN_DATA + "/agsal_49_clean.csv")
leaks_clean.to_csv(SAVE_CLEAN_DATA + "/agsal_full_clean.csv")

### Download leak patches

In [76]:
leaks_clean = pd.read_csv(SAVE_CLEAN_DATA + "/agsal_full_clean.csv").loc[0:99]

In [52]:
# Create leak type subdirectories inside leak and nonleak
for tipo in leaks_clean["tipo_cod"].unique():
    if os.path.exists(download_path + f"{tipo}"):
        print(f"{tipo} dir exists! Skipping...")
        continue
    else:
        print(f"{tipo} dir does not exists! Making directory at {download_path + tipo}...")
        os.mkdir(download_path + f"{tipo}")

FP dir does not exists! Making directory at ../data/patches_raw_agsal/leak/FP...
FC dir does not exists! Making directory at ../data/patches_raw_agsal/leak/FC...
FB dir does not exists! Making directory at ../data/patches_raw_agsal/leak/FB...
TB dir does not exists! Making directory at ../data/patches_raw_agsal/leak/TB...
TE dir does not exists! Making directory at ../data/patches_raw_agsal/leak/TE...


In [55]:
# Initialize GEE API
try:
    ee.Initialize()
except:
    ee.Initialize()
    ee.Authenticate()


Successfully saved authorization token.


In [79]:
prev_rep = 0
for rep in list(range(25, len(leaks_clean) + 25, 25)):

    for leak in range(prev_rep, rep):
        print("="*100)
        print("Leak index: ", leak)
        date_leak = datetime.strptime(leaks_clean.fechalegal[leak], date_format).date()
        id_leak = leaks_clean.id[leak]
        id.append(id_leak)
        
        # Dates for patch

        end_date_landsat = date_leak - timedelta(days = 5)
        start_date_landsat = end_date_landsat - timedelta(days = 25)    

        print("Date of leak: ", date_leak)

        if leak == 0:
            previous_end_date = ""

        if previous_end_date == date_leak:
            i += 1
            print("Leak detected at the same date")
        else:
            i = 0

        # Date for zip label
        leak_type = leaks_clean.tipo_cod[i]
        print(f"Leak type: {leak_type}")
        date_lab = "i" + str(int(id_leak)) + "d" + "_" + leak_type + "_" + str(leaks_clean.fechalegal[leak])[:10] + "_" + str(leak)
    
        # Coords of leaks
        leak_lon = leaks_clean["y"][leak]
        leak_lat = leaks_clean["x"][leak]

        # Point of leak
        poi = ee.Geometry.Point([leak_lat, leak_lon])
 
        print("Leak coords: ", (leak_lat, leak_lon), "\n")

        # Get sentinel and landsat collection of images according to leak coord 
        img_landsat = get_landsat_images(start = start_date_landsat, end = end_date_landsat, poi_leak = poi, buffer_regression = 500)
        landsat_date = ee.Date(img_landsat.get("system:time_start")).format("yyyy-MM-dd").getInfo()
        landsat_date = datetime.strptime(landsat_date, "%Y-%m-%d").date()

        # Sentinel image is selected after using landsat image date in order to get the minimun time between images
        end_date_sentinel = landsat_date
        start_date_sentinel = end_date_sentinel - timedelta(days = 5)    

        img = get_image(start = start_date_sentinel, end = end_date_sentinel, poi_leak = poi)
        sentinel_date = ee.Date(img.get("system:time_start")).format("yyyy-MM-dd").getInfo()
        sentinel_date = datetime.strptime(sentinel_date, "%Y-%m-%d").date()
        
        # Append date of images
        sentinel_img_date.append(sentinel_date)
        landsat_img_date.append(landsat_date)

        # Get difference of days between dates
        date_diff.append(abs(sentinel_date - landsat_date))
        date_diff_leak.append(sentinel_date - date_leak)
        print("Difference between image dates:", abs(sentinel_date - landsat_date))
        print("Difference between date of leak and date of image:" , abs(sentinel_date - date_leak))

        # Append cloud %
        sentinel_cloud.append(img.get("CLOUDY_PIXEL_PERCENTAGE").getInfo())
        landsat_cloud.append(img_landsat.get("CLOUD_COVER").getInfo())

        # Select bands and reduce the hole image to a patch centered on the leak
        img = bands_clip_image(img, buffer_size = patch_size, bands = bands)

        # Estimate lst from landsat image and add its lst band to sentinel image
        lst_image, rmse_value = lst_regression(landsat_image = img_landsat, sentinel_image = img, poi_leak = poi, buffer_size = patch_size, buffer_regression = 500, error = True)
        rmse.append(rmse_value)
        img = add_lst10m_band(img, lst_image = lst_image)

        # Download image
        download_image(image = img, path = download_path + f"{leak_type}/", date_label = date_lab)

        previous_end_date = date_leak

        index_progress.append(leak)

    print("+"*100)
    print("PATCH DOWNLOAD COMPLETED", "Batch:", rep)
    prev_rep = rep
    winsound.Beep(2000, 1000)
    if prev_rep == 985:
        break


Leak index:  0
Date of leak:  2024-01-16
Leak type: FP
Leak coords:  (-100.9786398, 25.4389816) 

Date of selected image (Landsat): 2023-12-24 
Landsat images found:  4 
Cloud %:  8.29
Date of selected image (Sentinel):  2023-12-20 
Sentinel images found: 3 
Cloud %:  64.525414
Difference between image dates: 4 days, 0:00:00
Difference between date of leak and date of image: 27 days, 0:00:00
Download complete
Leak index:  1
Date of leak:  2024-01-16
Leak detected at the same date
Leak type: FC
Leak coords:  (-100.9784005, 25.4383149) 

Date of selected image (Landsat): 2023-12-24 
Landsat images found:  4 
Cloud %:  8.29
Date of selected image (Sentinel):  2023-12-20 
Sentinel images found: 3 
Cloud %:  64.525414
Difference between image dates: 4 days, 0:00:00
Difference between date of leak and date of image: 27 days, 0:00:00
Download complete
Leak index:  2
Date of leak:  2024-01-16
Leak detected at the same date
Leak type: FB
Leak coords:  (-101.0096951, 25.4544524) 

Date of select